In [221]:
import requests
import json
import pandas as pd
from datetime import datetime
from influxdb_client import InfluxDBClient

### Set the time for Now

In [222]:
now = datetime.utcnow()

### Get bearer token

In [223]:
username = "timothy_woodland@outlook.com"
password = "@j-tLtrR8F"
client_secret = 'wZaRN7rpjn3FoNyF5IFuxg9uMzYJcvOoQ8QWiIqS3hfk6gLhVlG57j5YNoZL2Rtc'

In [224]:
payload = {'client_id':'tado-web-app',
            'grant_type':'password',
            'scope':'home.user',
            'username':username,
            'password':password,
            'client_secret':client_secret}

In [225]:
token_r = requests.post('https://auth.tado.com/oauth/token', params=payload)
token = token_r.json()["access_token"]
# print(token)

### Set Header for future Requests

In [226]:
header = {'Authorization' : "Bearer " + token}

### Get Home ID

In [227]:
home_id_r = requests.get("https://my.tado.com/api/v1/me", headers = header)
home_id = home_id_r.json()['homeId']
# print(home_id)

### Get name of home

In [228]:
home_url = "https://my.tado.com/api/v2/homes/"+str(home_id)
home_r = requests.get(home_url , headers = header)

In [229]:
home = home_r.json()["name"]

### Get zone data

In [230]:
zones_list_url = home_url+"/zones"

In [231]:
zone_list_r = requests.get(zones_list_url , headers = header)

In [232]:
zone_list_json = zone_list_r.json()
# print(json.dumps(zone_list_json, indent=1))

In [233]:
zone_list = []
zone_dict = {}
counter = 1


for zone_data in zone_list_json:
    zone_id = zone_data["id"]
    zone_name = zone_data["name"]

    zone_list.append(zone_id)
    
    zone_dict[zone_id] = {"time":str(now), "zone":zone_name}

    counter = counter + 1

### Get data from each zone and put into a dataframe

In [234]:
for zone in zone_list:
    zone_url = zones_list_url+"/"+str(zone)+"/state"
    zone_r = requests.get(zone_url, headers = header)
    zone_json = zone_r.json()

    mode = zone_json["tadoMode"]
    geo_override = zone_json["geolocationOverride"]
    type = zone_json["setting"]["type"]
    status = zone_json["setting"]["power"]
    set_temp = zone_json["setting"]["temperature"]["celsius"]
    power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
    zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
    zone_humidity = zone_json["sensorDataPoints"]["humidity"]["percentage"]

    zone_data_dict = {"mode":mode, "geo_override":geo_override, "type":type, "status":status, "set_temp":set_temp, "power_level":power_level, "zone_temp":zone_temp, "zone_humidity":zone_humidity }

    zone_dict[zone].update(zone_data_dict)
    

In [235]:
zone_df = pd.DataFrame.from_dict(zone_dict, orient="index")

In [236]:
zone_df = zone_df.reset_index()
zone_df= zone_df.rename(columns={"index":"zone_id"})

In [237]:
zone_df = zone_df.set_index("time")

### Get Weather data and put into a dataframe

In [238]:
weather_url = "https://my.tado.com/api/v2/homes/"+str(home_id)+"/weather"

In [239]:
weather_r = requests.get(weather_url , headers = header)

In [240]:
weather_json = weather_r.json()

In [241]:
weather_data = [weather_json["outsideTemperature"]["celsius"], weather_json["weatherState"]["value"], "tado"]

In [242]:
weather_columns = ["temperature_C", "state", "source"]

In [243]:
weather_df = pd.DataFrame(data=[weather_data], index=[now], columns=weather_columns)

### Write Zone and Weather dataframes to Influx db

In [244]:
# db_url = "http://192.168.10.172:8086"
# with InfluxDBClient(url=db_url) as ifdb_client:
#     with ifdb_client.write_api() as ifdb_write_client:

#         ifdb_write_client.write("testingdb", org="", record=weather_df, data_frame_measurement_name="tado_weather_data", data_frame_tag_columns=["source"])

In [245]:
db_url = "http://192.168.10.172:8086"
with InfluxDBClient(url=db_url) as ifdb_client:
    with ifdb_client.write_api() as ifdb_write_client:

        ifdb_write_client.write("testingdb", org="", record=weather_df, data_frame_measurement_name="tado_weather_data", data_frame_tag_columns=["source"])

        ifdb_write_client.write("testingdb", org="", record=zone_df, data_frame_measurement_name="tado_zone_data", data_frame_tag_columns=["zone_id", "zone", "mode", "geo_override", "type", "status"])